In [1]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [2]:
import pandas as pd
import plotly.graph_objects as go
import dash
import dash.html as html  # Import dash.html instead of dash_html_components
import dash.dcc as dcc  # Import dash.dcc instead of dash_core_components
from dash.dependencies import Input, Output

In [3]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2025-05-09 23:22:33--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.42K  --.-KB/s    in 0s      

2025-05-09 23:22:34 (611 MB/s) - ‘spacex_launch_dash.csv’ saved [2476/2476]



In [4]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/t4-Vy4iOU19i8y6E3Px_ww/spacex-dash-app.py"

--2025-05-09 23:22:34--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/t4-Vy4iOU19i8y6E3Px_ww/spacex-dash-app.py
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2075 (2.0K) [application/x-python]
Saving to: ‘spacex-dash-app.py’

spacex-dash-app.py  100%[===================>]   2.03K  --.-KB/s    in 0s      

2025-05-09 23:22:35 (898 MB/s) - ‘spacex-dash-app.py’ saved [2075/2075]



In [5]:
#!python3.11 spacex-dash-app.py

In [6]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

import pandas as pd
import dash
import dash.html as html  # Import dash.html instead of dash_html_components
import dash.dcc as dcc  # Import dash.dcc instead of dash_core_components
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv") # Changed the file name here
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                    options=[{'label': 'All Sites', 'value': 'ALL'}] + [
                                        {'label':l, 'value':l}
                                        for l in spacex_df['Launch Site'].unique().tolist()
                                    ],
                                    value="ALL",
                                    placeholder="Select a Launch Site here",
                                    searchable=True
                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                    min=0,
                                    max=10000,
                                    step=1000,
                                    value=[min_payload, max_payload]
                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        data = spacex_df
        fig = px.pie(data, values='class', names='Launch Site',
                title='Total Success Launches By Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        data = (spacex_df
            .loc[spacex_df['Launch Site'] == entered_site]
            .groupby('class')[['class']]
            .count()
            .rename({"class":"class_count"}, axis=1)
            .reset_index()
        )
        fig = px.pie(data, values='class_count', names='class',
                title=f'Total Success Launches for site "{entered_site}"')
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [
                  Input(component_id='site-dropdown', component_property='value'),
                  Input(component_id='payload-slider', component_property='value')
              ])
def get_scatter_plot(entered_site, entered_payload):
    min_value = entered_payload[0]
    max_value = entered_payload[1]
    data = spacex_df.loc[
        (spacex_df['Payload Mass (kg)'] >= min_value) &
        (spacex_df['Payload Mass (kg)'] <= max_value)
    ]
    if entered_site == 'ALL':
        fig = px.scatter(data, x="Payload Mass (kg)", y="class", color="Booster Version Category",
                title=f'Correlation between Payload and Success for all Sites')
        return fig
    else:
        data = data.loc[data['Launch Site'] == entered_site]
        fig = px.scatter(data, x="Payload Mass (kg)", y="class", color="Booster Version Category",
                title=f'Correlation between Payload and Success for site "{entered_site}"')
        return fig


# Run the app
if __name__ == '__main__':
    app.run() # Changed from app.run_server() to app.run()

--2025-05-09 23:22:35--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: ‘spacex_launch_dash.csv.1’

spacex_launch_dash. 100%[===================>]   2.42K  --.-KB/s    in 0s      

2025-05-09 23:22:35 (970 MB/s) - ‘spacex_launch_dash.csv.1’ saved [2476/2476]



<IPython.core.display.Javascript object>